In [6]:
import glob
import os
import pandas as pd
import polars as pl
from itertools import zip_longest
import s3fs
import ast 
import csv
import re
from Bio.Seq import Seq

import sys
sys.path.append('../')
from src.functions import import_s3_folder, upload_file_s3, del_file_s3
from src.protein_manip import dna_to_aa

ModuleNotFoundError: No module named 'src'

In [3]:
s3_folder = 'KEGG_db/genomes'

In [34]:
#for filename in glob.glob(os.path.join('/home/onyxia/work/KEGG_Pipeline/newly_filled/' , "*.parquet")):
#
#    df = pl.read_parquet(filename)
#    upload_file_s3(s3_folder, filename )

Uploaded gamer35/KEGG_db/genomes/pant_genome.parquet
Uploaded gamer35/KEGG_db/genomes/lkf_genome.parquet
Uploaded gamer35/KEGG_db/genomes/rbn_genome.parquet
Uploaded gamer35/KEGG_db/genomes/psin_genome.parquet
Uploaded gamer35/KEGG_db/genomes/sene_genome.parquet
Uploaded gamer35/KEGG_db/genomes/smt_genome.parquet
Uploaded gamer35/KEGG_db/genomes/nfl_genome.parquet
Uploaded gamer35/KEGG_db/genomes/mvz_genome.parquet


In [3]:
s3_folder = 'KEGG_db/genomes'
local_path = '/home/onyxia/work/KEGG_Pipeline/data/'
#import_s3_folder(s3_folder, local_path, format = 'parquet')

In [26]:
df = pl.read_parquet('/home/onyxia/work/KEGG_Pipeline/newly_filled/alkg_genome.parquet')

In [27]:
df

shape: (416, 4)
┌────────────────────────┬────────────────────────┬────────────────────────┬───────────────────────┐
│ gene_id                ┆ K0                     ┆ Dna_SEQ                ┆ Prot                  │
│ ---                    ┆ ---                    ┆ ---                    ┆ ---                   │
│ list[str]              ┆ list[str]              ┆ list[str]              ┆ list[bool]            │
╞════════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════╡
│ ["alkg:MOJ78_00005",   ┆ ["K02313", "K02338", … ┆ ["atggagaatattaacgattt ┆ [true, true, … false] │
│ "alkg:MOJ78…           ┆ ""]                    ┆ atggaataaa…            ┆                       │
│ ["alkg:MOJ78_00055",   ┆ ["", "", … "K01736"]   ┆ ["tctggtgacgacggcgaaga ┆ [false, true, … true] │
│ "alkg:MOJ78…           ┆                        ┆ ggtcacaccc…            ┆                       │
│ ["alkg:MOJ78_00105",   ┆ ["K00817", "K04517", … ┆ ["atgaaggtgaaaaaacagtt ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ ""]                    ┆ gttgtcacta…            ┆                       │
│ ["alkg:MOJ78_00155",   ┆ ["K01990", "K01992", … ┆ ["atgagtgatacatatgcaat ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ "K02033"]              ┆ cgagttgaaa…            ┆                       │
│ …                      ┆ …                      ┆ …                      ┆ …                     │
│ ["alkg:MOJ78_20605",   ┆ ["K04096", "K00783", … ┆ ["ttgaatataaaaaatactct ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ ""]                    ┆ gttgctttta…            ┆                       │
│ ["alkg:MOJ78_20655",   ┆ ["", "", … "K03111"]   ┆ ["ggctcgatagctcagtcggt ┆ [false, false, …      │
│ "alkg:MOJ78…           ┆                        ┆ agagcagagg…            ┆ true]                 │
│ ["alkg:MOJ78_20705",   ┆ ["K02990", "K06942", … ┆ ["atgcgtagatatgaaatcat ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ "K03496"]              ┆ gtacattctg…            ┆                       │
│ ["alkg:MOJ78_20755",   ┆ ["K03497", "K03501", … ┆ ["atgaaacaaccgttttcacg ┆ [true, true, … true]  │
│ "alkg:MOJ78…           ┆ "K02914"]              ┆ tttgtttgga…            ┆                       │
└────────────────────────┴────────────────────────┴────────────────────────┴───────────────────────┘

Some of the genes that were extracted after the main genomes and exported directly in parquet don't exactly have the same type of data stored. Knowing that we condition the treatment of our dfs depending on that characteristic.

In [12]:
### Polars implementation ###

#path of the .parquet files
local_path = '/home/onyxia/work/KEGG_Pipeline/newly_filled/'

all_files = glob.glob(os.path.join(local_path , "*.parquet"))
li = []
null_dfs = []

genes_total = 0 
no_kos= 0

#iterating through each genome
for i, filename in enumerate(all_files):

    #reads the parquet file saved during scraping
    df = pl.read_parquet(filename, columns= ['gene_id', 'K0', 'Dna_SEQ', 'Prot'])

    #if the column is a list we give its type back (initially a problem when first scraped in .csv and then translated in parquet)
    if str(df.dtypes[0])!= 'List(Utf8)':
        for col in df.columns:
            df = df.with_columns(pl.col(col).map_elements(ast.literal_eval))

    #check the length of each row, most of them are 10
    for col in df.columns:
        df = df.with_columns((pl.col(col).map_elements(len)).alias(f"{col}_len"))

    #We filter the data so that we can correctly explode the dataframe
    df2 = df.filter(
    (pl.col('gene_id_len') == pl.col('K0_len')) &
    (pl.col('gene_id_len') == pl.col('Dna_SEQ_len')) &
    (pl.col('gene_id_len') == pl.col('Prot_len'))
)
    if df.shape[0] != df2.shape[0]:
        print(f'{df.shape[0] - df2.shape[0]} problem lines in {filename}')

    df2 = df2.select(['gene_id', 'K0', 'Dna_SEQ', 'Prot'])
    df2 = df2.explode(['gene_id', 'K0', 'Dna_SEQ', 'Prot'])

    genes_total += df2.shape[0]

    
    #calculer des statistiques descriptives ici
    
    df3 = df2.filter(pl.col('K0')!= '')
    df4 = df3.filter(pl.col('Prot')==True)

    df4 = df4.with_columns(pl.col('Dna_SEQ').map_elements(lambda x: str(Seq(x).translate(stop_symbol=''))).alias('AA_seq'))\
             .drop(['Dna_SEQ', 'Prot'])
    

    no_kos += df2.shape[0] - df3.shape[0]
    #find outliers/errors
    li.append(df4)

In [13]:
li

[shape: (367, 3)
 ┌──────────────────┬────────┬───────────────────────────────────┐
 │ gene_id          ┆ K0     ┆ AA_seq                            │
 │ ---              ┆ ---    ┆ ---                               │
 │ str              ┆ str    ┆ str                               │
 ╞══════════════════╪════════╪═══════════════════════════════════╡
 │ nanh:LR957_00005 ┆ K02316 ┆ MNDAKEEVRARLNIEDVIGEYVQLKRAGRNLK… │
 │ nanh:LR957_00025 ┆ K02335 ┆ MKRLVVIDGKSVFYRGYYAMPGLSMADGTPTG… │
 │ nanh:LR957_00035 ┆ K10563 ┆ MPELPEVETVRRGLADLLPGQAVVRATVFDSP… │
 │ nanh:LR957_00040 ┆ K02340 ┆ VIYLFYGENEFEKRQAITKLIGNEKAARHDGE… │
 │ …                ┆ …      ┆ …                                 │
 │ nanh:LR957_03620 ┆ K01887 ┆ MEQIISQVVKQLFDQDISVQLTRPDPKFGDFA… │
 │ nanh:LR957_03625 ┆ K03282 ┆ MVKPSIDKSAKKLASKSAKAATKVATIKEKKI… │
 │ nanh:LR957_03630 ┆ K09903 ┆ MTKRILLKLSGEQLQGEFASGFDPKRARWIAE… │
 │ nanh:LR957_03635 ┆ K13288 ┆ MKASFKPKKILWVDLEMTGLDPVRDEILEVAA… │
 └──────────────────┴────────┴───────────────

In [59]:
del li[5596]

In [14]:
df = pl.concat(li)


In [15]:
df

gene_id,K0,AA_seq
str,str,str
"""nanh:LR957_000…","""K02316""","""MNDAKEEVRARLNI…"
"""nanh:LR957_000…","""K02335""","""MKRLVVIDGKSVFY…"
"""nanh:LR957_000…","""K10563""","""MPELPEVETVRRGL…"
"""nanh:LR957_000…","""K02340""","""VIYLFYGENEFEKR…"
"""nanh:LR957_000…","""K03177""","""MDEVIIIDKPQGMT…"
"""nanh:LR957_000…","""K02956""","""MISKENKAKAIALT…"
"""nanh:LR957_000…","""K07284""","""MSLQLDSVKNRQKS…"
"""nanh:LR957_000…","""K00962""","""MSIINPSGKEIFSV…"
"""nanh:LR957_000…","""K21929""","""MDKQVKLDALAEEI…"


In [20]:
df = df.select(['gene_id', 'K0', 'Dna_SEQ'])

In [21]:
df

gene_id,K0,Dna_SEQ
str,str,str
"""pfi:PFC_00005""","""K02035""","""atgaggaaaaaact…"
"""pfi:PFC_00010""","""K02033""","""atggggttaggtaa…"
"""pfi:PFC_00015""","""K02034""","""atgagatgggtaga…"
"""pfi:PFC_00020""","""K02031""","""atggctaagaagat…"
"""pfi:PFC_00025""","""K02032""","""atggccgagccaat…"
"""pfi:PFC_00035""","""K06859""","""atgtataaggaacc…"
"""pfi:PFC_00040""","""K23264""","""atgagatggagggt…"
"""pfi:PFC_00045""","""K23265""","""atggtaaaatttgc…"
"""pfi:PFC_00050""","""K23269""","""atgtttccccatga…"


In [22]:
local_path = '/home/onyxia/work/KEGG_Pipeline/data/Prok_proteins.parquet'
df.write_parquet(local_path)

In [23]:
s3_folder = 'KEGG_db'

In [24]:
upload_file_s3('KEGG_db', local_path)

Uploaded gamer35/KEGG_db/Prok_proteins.parquet


In [17]:
df["K0"].value_counts().sort(by = pl.col('counts'))

K0,counts
str,u32
"""K16117""",1
"""K07374""",1
"""K20764""",1
"""K21319""",1
"""K12704""",1
"""K00109""",1
"""K21168""",1
"""K15872""",1
"""K12622""",1


In [16]:
no_kos

298105

In [17]:
genes_total

556919

In [17]:
import os
import s3fs

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
local_path = '/home/onyxia/work/KEGG_Pipeline/newly_filled/'
s3_folder="gamer35/KEGG_db/retardataires/"

In [18]:
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},key ='HS2RC8GA5SH4XX52N0L8', secret = 'QC15sHsrqHuDdZYgfUj1lbbqA7Q+x9Oa4oGhOVSh', token = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJIUzJSQzhHQTVTSDRYWDUyTjBMOCIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzA2NTM1NTEzLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImZsb3JpYW4uYmFzc29AZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzA2NjI0MzQxLCJmYW1pbHlfbmFtZSI6IkJhc3NvIiwiZ2l2ZW5fbmFtZSI6IkZsb3JpYW4iLCJncm91cHMiOlsiVVNFUl9PTllYSUEiXSwiaWF0IjoxNzA2NTM3OTQwLCJpc3MiOiJodHRwczovL2F1dGgubGFiLnNzcGNsb3VkLmZyL2F1dGgvcmVhbG1zL3NzcGNsb3VkIiwianRpIjoiZjI2Zjk1YjYtZmExZS00NmVmLTk1ZDgtNWZlMGVmMGYzZGY3IiwibmFtZSI6IkZsb3JpYW4gQmFzc28iLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiZ2FtZXIzNSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2Vzc2lvbl9zdGF0ZSI6ImVhNGY4NWMyLWNlNDgtNGJkOS05ZGFhLWIwMjY2MjRiOTBjOSIsInNpZCI6ImVhNGY4NWMyLWNlNDgtNGJkOS05ZGFhLWIwMjY2MjRiOTBjOSIsInN1YiI6IjVmY2JjNGE5LThmNzItNDkzMy05MDJmLTNiYWNlMjMyNjk2MSIsInR5cCI6IkJlYXJlciJ9.vztHXgGJbdoSn0BMwxtvRHoDfTIQ63XwQqUiReKWgUbSYMaQl4XRGBznL5X5Dc3ue2l3yQ7LstZcv064HSoLEw')
fs.ls("gamer35")

['gamer35/KEGG_database',
 'gamer35/KEGG_db',
 'gamer35/Statapps_google',
 'gamer35/public',
 'gamer35/shells_scripts']

In [19]:
fs.put("/home/onyxia/work/KEGG_Pipeline/bonjour.txt", s3_folder)

In [4]:
for file in os.listdir(local_path):
    #s3fs.put(file,)
    print(file)

DB.lookup
KEGG_Pathways.csv
DB.source
DB_h.dbtype
Prok_proteins.fasta
DB
DB_h
.ipynb_checkpoints
tmp
truncated_proteins.fasta
Prok_proteins.parquet
DB.dbtype
DB_h.index
empty.txt
DB.index


In [21]:
local_path

'/home/onyxia/work/KEGG_Pipeline/newly_filled/'

In [24]:
for root, dirs, files in os.walk(local_path):
        # Print the path of each file
        for file in files:
            file_path = os.path.join(root, file)
            print(file_path)
            fs.put(file_path, s3_folder)
        
            


/home/onyxia/work/KEGG_Pipeline/newly_filled/nanh_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/ttg_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/cpyr_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/pbon_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/aog_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/pok_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/dzo_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/pthe_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/braz_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/phk_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/fhm_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/mefw_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/skg_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/mver_genome.parquet
/home/onyxia/work/KEGG_Pipeline/newly_filled/pspn_genome.parquet
/home/onyxia/work/KEGG_Pipeline/

**Genomes to correct**

genomes = ['pant', 'lkf','rbn','psin','sene','smt','nfl','mvz']

In [43]:
for i, df in enumerate(li):
    if str(df.dtypes[2])!= 'Utf8':
        print(i)

5596


In [46]:
del li[5596]

In [45]:
all_files[5596]

'/home/onyxia/work/KEGG_Pipeline/data/rbn_genome.parquet'

In [ ]:
df2

In [41]:
df2.dtypes[2]

Utf8

In [36]:
df2

('vta:A0001',
 'K01873',
 'atggaaaagacatacaacccaacatcaatcgaacaagctctatatcaagcttgggaagagaaaggctactttaagccacacggtgacacatcaaaagaagcttacagcatcatgataccgccaccgaacgtcactggtagcctgcatatgggccacgcgttccaagacaccatcatggatacgcttgtccgtgcagagcgtatgaagggtaaaaacacgctttggcaagtcggtactgaccacgcaggtatcgcgactcaaatggttgttgagcgcaaaattgctgcggaagaaggcaaaacaaaacacgactacggccgtgaagccttcatcgataaaatttgggaatggaaagctgaatctggtggcaccattacgaaacaacttcgtcgccttggcgcatcagtagattgggaacgtgagcgtttcacaatggatgatggcttatcagcagccactcaagaagttttcgtacgtttgtttgaagacgacctaatttaccgtggaaagcgcttggtaaactgggatccaaaacttcacaccgcgatctctgatcttgaagttgaaaacaaagacaaaaaaggcttcatgtggcacttccgctacccactagcgaacggtgttaaaaccgctgatggtaaagatcacatcgttgttgcaacgactcgtccagaaaccatgcttggtgatactggcgttgcagttaaccctgaagatccacgctacaaagatcttatcggtaaagagatcctacttcctatcgttaatcgtcttatccctatcttaggtgatgaacacgcagatatggaaaaaggcacaggttgtgtgaaaattacacccgctcatgactttaacgattacgaagtgggtaaacgcaataacctgccaatgatcaacatcctaaccttcaatgccgatatccgtgatgctgctgaaatattcacgaccaacggtgaagaaagtgatgtttact

In [32]:
str(df.dtypes[1])== 'List(Utf8)'

True

In [10]:
df

non_matching_rows = df.filter(
    (pl.col('gene_id_len') != pl.col('K0_len')) |
    (pl.col('gene_id_len') != pl.col('Dna_SEQ_len')) |
    (pl.col('gene_id_len') != pl.col('Prot_len'))
)

# Display the rows where the columns do not match
non_matching_rows

gene_id,K0,Dna_SEQ,Prot,gene_id_len,K0_len,Dna_SEQ_len,Prot_len
list[str],list[str],list[str],list[bool],i64,i64,i64,i64
"[""pant:PSNIH1_11520"", ""pant:PSNIH1_11525"", … ""pant:PSNIH1_11565""]","[""K09016"", ""K09024"", … """"]","[""atggcgagtacctggtttccacgctggcgcgttaagtcagcaatgaccgacgacggcttgattgccccggacgagacgctgccgctcgggcagacgctggtgcttgggctgcagcatgcggtcgcaatgtttggcgcaacggtactgatgccgctgctgatggggctggatgccaatctcgctgtgttactctccgggatcggtacgctgctgttcttttttattaccggcgggcgcgtaccgagctatctcggttccagcgccgcttttgtgggcgtagtgattgcggtcaccggtttcaccggccagggccttaacactcacctctccgtggcgctgggcggcattatcgcctgcggcgcgctctataccctgattggtctgattgtgatgaaagtgggtaccggctggattgagcgcctgatgccgccggtcgtcaccggcgccgtcgttatggcgattggcctgaacctggcgccgattgcggtgcacagtgtttctgccaccctgttcgacagctggatggcggtgctgacggtgctctgtatcggactggtggcggtcatgacgcgcgggctggtacagcgcctgttgatcctggtagggctgatcgtggcctgggcgttctatgcgctgctgaccaatggcttcggcctcggtaaaccggtggatttcagcggcgtggcgcacgccgcctggtttggcgtgccgcatacgaccgcccctgtgttcgacctgcaggcgatagtgatgattgcgccggtcgccattattctggtcgctgaaaatctggggcatattaaagcggtagcagctatgaccggacgcaatctcgatgcgtggatgggccgggcgtttgttggcgacggtctggccactatgctctctggctctgccggcggcagcggcgtgacaacctatgccgaaaacataggcgtcatggcggttacgcgcgtctactctacgctggcgttcgtcgccgcggccctgatcgcaatactgctgggtttttcaccgaaatttggcgcactcatccataccgtgccagcaccggtcataggcggcgcgtctatcgtggtatttggcctgattgcggtggccggtgcccgtatctggctgcagaatcaggtcgatctcgggcagaacggtaatctgatcatggtcgctaccacgctggttctgggcgccggcgattttgcgcttaagatcggcagcttcacgctcggcggtatcggcaccgcgacctttggcgccattatcctgaacgcaattctgcagcaccgtagccgcgcgccggccggttcagctgcacagccacacggttaa"", ""atgagcctacatgcccttccggatttacccgcagtcgatcgcgacacctttcgcaacgctatggcgcagctgggtgcagccgttaatgttatcaccaccgatggcccggcaggcgtggctggctttaccgcctccgccgtctgcagcgtgaccgactccccgccgacgctgctggtctgcctgaaccgctccgcctcagcctggccggcgtttcgcgataaccgtgcactctgcgtcaataccctctcggcagaccagcacgccctctccacgctgttcgggggcaaaaccgcgatggcggatcgctttgccgccgcgcagtggcagacgcttgccagcggtgctccgctgctggaggccgcgctgatctcctttgactgcgagataacacagcgggtcagcgttggcacacacgatattctgttctgtacggtgctggccgtggcccagggcgaggcgcgccaggggctcgcctggttcgatcgcggttatcacaccttctcccggcaggatgcccgctga"", … ""gtgcgcgcactcattgcgacaggaataattgctctggtggggtggtggagtttgccctggctgaaatcgcagcttccggcgcagtggaatccgtttattccgcttaacgtcacggatgcgccaggctggataacacgttacaagctggagaaactgcgtcacgatccagccgcctgtctggcggtgctggagcgcgccagtgcgcagggacgggtaagttttcgcacgacgcctccggtgcggggtaactgcacaattgaacagccggttcgtatcagccagtttgggccggttacgctcagcagcagttttcttgccagctgtccgatggccgtggcaagtacgatgtatgtcacgcgcagtaatgctgcattacagcaggcgggtatcaccagtccgctgcggcgtattgagcatgttggcagctacgcctgccgcaacgtctatcatcgccaggagggacgccgcagcgaacacgctacggcagaggcgtgggatgtcaccgcttttcagctggccgatggccgctggttacgggtcgccagaaactggcagcagcccgccagcagcgcggcgggtttgcatgcactctggcgcaacggctgtgccagctttggaaacgcactgggaccagactacaacgcagctcataccacacactttcatctgggcatgcgcggcggtggattctgccgctga""]","[true, true, … true]",10,9,9,10


In [13]:
df.shape[0]

422

In [5]:
len(all_files)

8525

In [ ]:
df_exploded.filter( df['KO']!= ""

In [6]:
len(li)

797

In [ ]:
len(null_dfs)